In [ ]:
# must download when new kernel created
%pip install git+https://github.com/huggingface/transformers -qqq
%pip install bitsandbytes -qqq
%pip install huggingface_hub -qqq
%pip install peft -qqq
%pip install accelerate -qqq
%pip install trl -qqq
%pip install wandb -qqq
%pip install ipywidgets -qqq

%pip install torch==2.1.0 torchvision torchaudio 

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ongjjbrandon/mistral-pjf-v3/model-gaflufph:v0', type='model')
artifact_dir = artifact.download()

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login
base_model, adapter, new_model = "mistralai/Mistral-7B-v0.1" , "./artifacts/model-gaflufph:v0/", "ogbrandt/bfloat-16-merge"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cpu")
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()
model.save_pretrained("/home/ubuntu/remote-training/merged_models")

In [12]:
# better merge?
# Clear the memory footprint
import torch

# del model
# torch.cuda.empty_cache()

# Reload the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model_reload, adapter)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [16]:
# from huggingface_hub import login
# login()

model.push_to_hub(new_model, use_temp_dir=True)
tokenizer.push_to_hub(new_model, use_temp_dir=True)

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ogbrandt/bfloat-16-merge/commit/21fda3694ea3ab5cfe859bb89faf0f72f3829af6', commit_message='Upload tokenizer', commit_description='', oid='21fda3694ea3ab5cfe859bb89faf0f72f3829af6', pr_url=None, pr_revision=None, pr_num=None)